In [1]:
import pandas as pd
import os
import sys
import numpy as np

## Chargement des données

In [2]:
src_path = os.path.dirname(os.path.realpath('__file__'))
data_path = os.path.join(os.path.dirname(src_path), 'data')

name_basics_file = os.path.join(data_path, 'name.basics.tsv')
title_akas_file = os.path.join(data_path, 'title.akas.tsv')
title_basics_file = os.path.join(data_path, 'title.basics.tsv')
title_principals_file = os.path.join(data_path, 'title.principals.tsv')
title_ratings_file = os.path.join(data_path, 'title.ratings.tsv')

# df_name_basics = pd.read_table(name_basics_file, nrows=100000)
# df_title_akas = pd.read_table(title_akas_file, nrows=100000)
# df_title_basics = pd.read_table(title_basics_file, nrows=100000)
# df_title_principals = pd.read_table(title_principals_file, nrows=100000)
# df_title_ratings = pd.read_table(title_ratings_file, nrows=100000)

In [6]:
df_title_ratings = pd.read_table(title_ratings_file)

## Question 1: La liste des acteurs par film

In [3]:
# get only people who have job 'actor' and 'actress'
df_principals = df_title_principals.loc[df_title_principals['category'].isin(['actor', 'actress'])].filter(['tconst', 'nconst']).drop_duplicates()

# merge principals with actors
df_principals_actor_assoc = pd.merge(df_principals, df_name_basics, how='inner', on='nconst')[['tconst', 'nconst', 'primaryName']].drop_duplicates()

# merge films with actors
df_principals_film_actor_assoc = pd.merge(df_principals_actor_assoc, df_title_basics, on='tconst')[['originalTitle', 'primaryName']].drop_duplicates()

# group actors by film
df_actors_groupes_by_films = df_principals_film_actor_assoc.groupby(['originalTitle'])['primaryName'].apply(','.join).reset_index()

## Question 2: La liste des films Américains (en gardant leur nom en français) et leur note moyenne

In [22]:
# get only movies distriuted in US
df_akas_us = df_title_akas.loc[df_title_akas['region'] == "US"].filter(['titleId', 'title', 'region'])
df_akas_us.rename(columns = {'title': 'title_us', 'region': 'region_us'}, inplace = True)

# get only movies distriuted in FR
df_akas_fr = df_title_akas.loc[df_title_akas['region'] == "FR"].filter(['titleId', 'title', 'region'])
df_akas_fr.rename(columns = {'title': 'title_fr', 'region': 'region_fr'}, inplace = True)

# merge movies distributed in US and in FR
df_akas_merged = pd.merge(df_akas_us, df_akas_fr, how='left', on='titleId')

# create new column "is_title_fr" to describe if movie has english title or only french title
df_akas_merged['is_title_fr'] = np.where(df_akas_merged['title_fr'].isnull(), False, True)

# remove US title if FR title exists
df_akas_merged['title_us'] = np.where(df_akas_merged['title_fr'].notnull(), '', df_akas_merged['title_us'])

# merge with rating
df_rating_akas = pd.merge(df_akas_merged, df_title_ratings, how='inner', right_on='tconst', left_on='titleId')

# remove columns
df_rating_akas = df_rating_akas.filter(['titleId', 'title_fr', 'title_us', 'is_title_fr', 'averageRating'])

## Question 3: Les notes moyennes des différents genres

In [23]:
# split column gere by ','
new_df_dropna = df_title_basics[['tconst', 'genres']].dropna()
new_df_filtred = new_df_dropna[new_df_dropna['genres'] != '\\N']

new_df = pd.DataFrame(new_df_filtred.genres.str.split(',').tolist(), index = new_df_filtred.tconst).stack()

# define 'tconst' as index
new_df = new_df.reset_index([0, 'tconst'])

# rename columns
new_df.columns = ['tconst', 'genre']

# merge new df with rating dataframe
merge_genre_rating = pd.merge(new_df, df_title_ratings, how = 'inner', on = 'tconst')

# keep only titles 'genre' and 'averageRating'
merge_genre_rating = merge_genre_rating.filter(['genre', 'averageRating'])

# calcul of average by genre
avg_rating_genre = merge_genre_rating.groupby('genre').mean('averageRating')

# round average rating to 2 decimals
avg_rating_genre = avg_rating_genre['averageRating'].round(decimals=2)

## Bonus 1: Générer des recommandations

In [55]:
## create a dataset of users
users = []
user = []
import random

movie_list = df_title_basics['tconst'].to_list()

df = pd.DataFrame(columns=['userId', 'tconst', 'averageRating'])
index = 0

for i in range(1, 11):
    for j in range(1, 21):
        index += 1
        df.loc[index, 'userId'] = i
        df.loc[index, 'tconst'] = random.choice(movie_list)
        df.loc[index, 'averageRating'] = format(random.uniform(3, 9), '.1f')
        
df.to_csv('../data/users.tsv', sep ='\t')
    

In [6]:
src_path = os.path.dirname(os.path.realpath('__file__'))
data_path = os.path.join(os.path.dirname(src_path), 'data')

title_basics_file = os.path.join(data_path, 'title.basics.tsv')
title_ratings_file = os.path.join(data_path, 'title.ratings.tsv')
users_file = os.path.join(data_path, 'users.csv')

df_title_basics = pd.read_table(title_basics_file, nrows=1, usecols = ['tconst', 'primaryTitle'])
df_title_ratings = pd.read_table(title_ratings_file, nrows=1)
df_users = pd.read_csv(users_file)


In [ ]:
# Chargement des movies avec leurs raiting et vote
